# Erste Experimente mit Numba
- Voxelfilter: außen parallel, innen nopython
- Compilierung 8 s
- Dann Laufzeit auf ein Drittel
- Worker -1 in kdtree.query lohnt sich (10 mal scheller, von 0.1 zu 0.01)
- Jupyter Notebook 
    - ist laut ChatGPT schon multithreaded
    - Keine genaue Vergleichbarkeit mit cpython
    - Evtl. dauert Compilierung in Jupyter etwas länger als bei cpython

In [13]:
import pdal 
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import os
import json
import time
from scipy.spatial import KDTree
from scipy.linalg import svd 
import open3d as o3d
from osgeo import ogr

from numba import jit, njit, prange

from interessant import * # Bei Änderungen Kernel neu starten
ogr.UseExceptions()


In [ ]:
run = run24
#run = run14
# filename = interessant['OLA gleiche Höhe wie Gleis']

# Bahnsteig: 29; Gleis hohe Intensität: 11; Weiche B: 16; Unterirdischer Bhf: 20; Gleis weit abseits: 23; Betondeckel: 28; Zug run 14 A (in run24 Achszähler): 6; 
# Viele Gleise: 33; Anfang Weiche: 34, OLA gleiche H: 35; Y: 37
key = list(interessant.keys())[33] 
filename = interessant[key]
print(key, filename)

filename = os.path.join(run, filename)
if not os.path.exists(filename):
    raise FileNotFoundError(filename)

Viele Gleise 4474750_5332150.copc.laz


In [15]:
empty_space_thresh = 8  # z.B. 5 oder 8
majority_tresh  = 0.5 # Erster Durchgang 0.3, bei "Gleis hohe Intensität" gibt 0.5 ein viel besseres Ergebnis
ground_percentile = 10 # 10 in Oude E.
top_percentile = 99.5 # 90 in Oude E.

voxel_size = 1.0

voxel_size = 25 / 30
print("Voxel size:", voxel_size)

minimum_points = 50 # Erste Versuche mit 100, aber viel schwarz bei abseits liegenden Gleisen. 50 ist besser.

with_normals = False

Voxel size: 0.8333333333333334


In [16]:
pipeline = pdal.Pipeline([pdal.Reader(filename)])
pipeline.execute()
points = pipeline.arrays[0]

points['Classification'] = 0
points.shape

(2289989,)

In [17]:
intensity = points['Intensity']
intensity_normalized = (intensity - intensity.min()) / (intensity.max() - intensity.min())
colormap = plt.get_cmap("viridis")
intensity_colors = colormap(intensity_normalized)
intensity_colors = intensity_colors[:, :3]

- ohne jit: 0.05 s
- jit: funktioniert nicht

In [18]:
# @jit
def get_xyz(points):
    xyz = np.vstack((points['X'], points['Y'], points['Z'])).transpose()

    # Offset entfernen (aber gerundet, damit Kachelgrenzen ganze Zahlen bleiben)
    offset = xyz.mean(axis=0).round() 
    xyz -= offset
    return xyz, offset

start = time.time()
xyz, offset = get_xyz(points)
end = time.time()
print(end - start)

0.04269814491271973


## Voxelfilter

In [19]:
RAIL = 20

maxp = xyz.max(axis=0)
minp = xyz.min(axis=0)
maxp, minp

(array([ 8.99998766,  5.99997468, 15.44354109]),
 array([-16.00001234, -18.99982532,   0.10734109]))

In [20]:
voxels = ((xyz[:, :2] - minp[:2]) // voxel_size).astype(int)
v_max = voxels.max(axis=0)
z = xyz[:, 2]

Funktioniert ursprünglich nicht wegen None und try-except

In [21]:
@njit
def check_constraints(z_values):
    foo = 0.3  
    # Threshold on number of points in voxel
    if len(z_values) < minimum_points:
        return np.array([False]) # None geht nicht mit Numba njit

    ground_level = np.percentile(z_values, ground_percentile) 
    # 10% Percentile
    # Check that there are almost no points 0.5 to 4.5 m above the ground
    # But allow for some noise

    count = ((z_values > ground_level + foo) & (z_values < ground_level + 4.5)).sum()

    if count <= empty_space_thresh:
        # Look for points within 0.5 m above ground and get 
        # 98 percentile (Oude E.) or better 99.5 percentile
        mask = (z_values > ground_level) & (z_values < ground_level + foo)
        if len(z_values[mask]) == 0:
            return np.array([False])
        candidates_top = np.percentile(z_values[mask], top_percentile)

        # Oude Elberink require the height difference > 0.1 m
        # And mark only the points 10 cm below the top as rail point candidates
        if candidates_top - ground_level > 0.1:
            mask = (z_values > candidates_top - 0.1) & (z_values < candidates_top + 0.05)

            # Also make sure these are only a minority of the points (otherwise it's a slope)
            if mask.sum() < majority_tresh * len(z_values):  # z.B. 0.3
                return mask
    return np.array([False])

In [22]:
def classify(voxels, z, points, v_max):
    for i in range (v_max[0] + 1):
        for j in range(v_max[1] + 1):
            mask = (voxels[:,0] == i) & (voxels[:,1] == j)
            z_values = z[mask]
            mask2 = check_constraints(z_values)
            if mask2 is not None:
                indices = np.where(mask)[0]
                points['Classification'][indices[mask2]] = RAIL

In [23]:
@njit(parallel=True)
def jitclassify(voxels, z, points, v_max):
    for i in prange (v_max[0] + 1):
        for j in prange(v_max[1] + 1):
            mask = (voxels[:,0] == i) & (voxels[:,1] == j)
            z_values = z[mask]
            mask2 = check_constraints(z_values)
            if mask2 is not np.array([False]):
                indices = np.where(mask)[0]
                points['Classification'][indices[mask2]] = RAIL

- ohne jit 3.8, 3.6, 3.9
- mit jit, nicht parallel 
    - erstes mal 4.6; 
    - dann 3.3, 3.4
- parallel: 
    - erste mal 7.88 oder 4.1!
    - dann 1.33, 1.4, 1.36

In [24]:
# start = time.time()

# classify(voxels, z, points, v_max)

# end = time.time()
# print(end - start)

In [25]:
start = time.time()

jitclassify(voxels, z, points, v_max)

end = time.time()
print(end - start)

4.125516414642334


## Gleispaar

In [26]:
tmpdir = "/media/riannek/minimax/gleis/temp_fertig"

filename = interessant[key]
filename = filename.split(".")[0] + ".ply"

if not os.path.exists(os.path.join(tmpdir, filename)):
    raise FileNotFoundError(filename)

In [27]:
thresh_side_distance = 0.15   # 0.1 ist meist gut, aber in manchen Fällen zu wenig (z.B. Anfang Weiche)

intensity_threshold = 14500
downsample_radius = 0.45 # 0.4
neighborhood_radius = 0.5 # 0.5

min_points = 10
minimum_in_hood = 10
linearity_tresh = 0.98

gauge = 1.435

In [28]:
gpkg = ogr.Open(os.path.join(tmpdir, "temp.gpkg"))
layer = gpkg.GetLayerByName("tiles")

In [29]:
#@njit # Lohnt sich nicht
def extend_bbox(bbox, margin=2):
    # MinX, MaxX, MinY, MaxY
    return (bbox[0] - margin, bbox[1] + margin, bbox[2] - margin, bbox[3] + margin)

def get_bbox_polygon(bbox):  
    ring = ogr.Geometry(ogr.wkbLinearRing)      
    ring.AddPoint_2D(bbox[0], bbox[2])  # MinX, MinY
    ring.AddPoint_2D(bbox[1], bbox[2])  # MaxX, MinY
    ring.AddPoint_2D(bbox[1], bbox[3])  # MaxX, MaxY
    ring.AddPoint_2D(bbox[0], bbox[3])  # MinX, MaxY
    ring.AddPoint_2D(bbox[0], bbox[2])  # Close ring
    geom = ogr.Geometry(ogr.wkbPolygon)
    geom.AddGeometry(ring)
    return geom

njit extend_bbox: 
- Compile 1 s
- Dann 0.003 s
Ohne njit genau gleich schnell!

In [30]:
start = time.time()

filter = f"filename = '{filename}'"
layer.SetAttributeFilter(filter)
feature = layer.GetNextFeature()
layer.SetAttributeFilter(None)
bbox = feature.GetGeometryRef().GetEnvelope()
extended = extend_bbox(bbox, margin=2)
bbox_geom = get_bbox_polygon(extended)
layer.SetSpatialFilter(bbox_geom)
tiles = [f.GetField("filename") for f in layer]
layer.SetSpatialFilter(None) 

print(tiles) 
print(time.time()- start)    

['4474725_5332175.ply', '4474750_5332175.ply', '4474750_5332125.ply', '4474725_5332150.ply', '4474775_5332125.ply', '4474775_5332150.ply', '4474750_5332150.ply', '4474725_5332125.ply', '4474775_5332175.ply']
0.8653550148010254


In [31]:
del gpkg

In [32]:
bounds = f"([{extended[0]}, {extended[1]}], [{extended[2]}, {extended[3]}])" 
bounds

'([4474748.0, 4474777.0], [5332148.0, 5332177.0])'

Allein das dauert 0.6 s

In [33]:
start = time.time()

readers = [pdal.Reader(os.path.join(tmpdir, tile)) for tile in tiles]
pipeline = pdal.Pipeline(readers) | pdal.Filter.merge() | pdal.Filter.crop(bounds=bounds)
pipeline.execute()
points = pipeline.arrays[0]
points.shape 

print(time.time()- start)

0.5804653167724609


### Seed

In [34]:
low_intensity = points[points["Intensity"] < intensity_threshold]
low_intensity.shape

(247472,)

In [35]:
bounds = f"([{bbox[0]}, {bbox[1]}], [{bbox[2]}, {bbox[3]}])" 
bounds

'([4474750.0, 4474775.0], [5332150.0, 5332175.0])'

In [36]:
# Downsample with poisson sampling (only original bbox)

downsampling_pipeline = pdal.Filter.crop(bounds=bounds).pipeline(low_intensity) | pdal.Filter("filters.sample", radius=downsample_radius)
downsampling_pipeline.execute()
seed_points = downsampling_pipeline.arrays[0]
seed_point_count = seed_points.shape[0]
seed_point_count

526

### KD Tree

In [37]:
xyz = np.vstack((points['X'], points['Y'], points['Z'])).transpose()
xyz_seed = np.vstack((seed_points['X'], seed_points['Y'], seed_points['Z'])).transpose()

In [38]:
offset = xyz.mean(axis=0).round() 
xyz -= offset
xyz_seed -= offset

In [39]:
start = time.time()
tree = KDTree(xyz)  
print(time.time()- start)

0.08271336555480957


- workers 1 (default) 0.4, 0.6, 0.7
- workers -1: 0.03, 0.03, 0.04
- lohnt sich!

In [40]:
# indices: ndarray (dtype object) with a list of indices for each seed point
start = time.time()
indices = tree.query_ball_point(xyz_seed, r=neighborhood_radius, workers=-1)
print(time.time()- start)

0.01767134666442871


In [41]:
def pca(cloud):
    """Use PCA to get einvalues and eigenvectors of a point cloud
    
    Returns (eigenvalues, eigenvectors)
    """
    if cloud.shape[0] < 3:
        raise ValueError("Point cloud must have at least 3 points")
    mean = np.mean(cloud, axis=0)
    centered = cloud - mean
    U, S, Vt = svd(centered, full_matrices=False)
    eigenvals = S**2/(cloud.shape[0]-1)
    # Returned vectors are in columns, first vector is eigenvec[:, 0] == eigenvec.T[0]
    return eigenvals, Vt.T

In [42]:
# @jit(forceobj=True)
def dbscan_stretchz(xyz, min_points=10, stretch=1.5):

    pointcount = xyz.shape[0]
    if pointcount <= min_points:
        return np.ones(pointcount, dtype=np.int8) * -1
    
    eps = 50 / pointcount
    eps = max(eps, 0.06)
    
    xyz = xyz.copy()
    xyz[:, 2] *= stretch
    hood_pcd = o3d.geometry.PointCloud()
    hood_pcd.points = o3d.utility.Vector3dVector(xyz) 
    # eps is min distance between clusters
    labels = np.array(hood_pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=False))
    return labels


In [43]:
@njit
def get_central_point(a, b):
    """Get the point in the middle of two points"""
    vector = b - a
    return a + vector / 2

In [44]:
@njit
def linearity(eigenvals):
    """Calculate the linearity of a point cloud"""
    return (eigenvals[0] - eigenvals[1]) / eigenvals[0]

In [45]:
# @jit(forceobj=True, looplift=True)
def cluster_generator(points, min_points):
    if points.shape[0] < min_points:
        return
    
    labels = dbscan_stretchz(points, min_points=min_points, stretch=4)
    
    max_label = labels.max()
    
    for label in range(max_label + 1):
        cluster = points[labels == label]
        if cluster.shape[0] < min_points:
            # Somehow this happens, must be a bug in open3d
            continue

        eigenvals, eigenvects = pca(cluster)
        cluster_linearity = linearity(eigenvals)


        if not ((cluster_linearity > linearity_tresh) and (eigenvals[0] > 0.04)):
            # Cluster not linear or too short
            continue

        clustercenter = cluster.mean(axis=0)

        # Make sure there are no points above the cluster
        # (exclude clusters at the base of the track profile)
        if not cluster_is_on_top(cluster, clustercenter, points):
            continue
        

        yield cluster, clustercenter, eigenvals, eigenvects

In [46]:
@njit
def cluster_is_on_top(cluster, clustercenter, points):
    """Check that there are no points above the cluster center"""
    x_min, x_max = clustercenter[0] - 0.1, clustercenter[0] + 0.1
    y_min, y_max = clustercenter[1] - 0.1, clustercenter[1] + 0.1
    z_min = cluster[:,2].max()

    mask = ((points[:, 0] > x_min) & 
        (points[:, 0] < x_max) & 
        (points[:, 1] > y_min) & 
        (points[:, 1] < y_max) & 
        (points[:, 2] > z_min))
    
    
    return points[mask].shape[0] == 0

In [47]:
# @jit(forceobj=True, looplift=True)
def check_cluster_for_railpair(clustercenter, eigenvects, pointcloud, tree, gauge, min_points):

    # Check for rail pair
    sidevector = np.cross(eigenvects[:,0], np.array([0, 0, 1])) * (gauge + 0.07) # Add railtop width to gauge
    bothsides = np.array([clustercenter + sidevector, clustercenter - sidevector])

    indices_bothsides = tree.query_ball_point(bothsides, r=neighborhood_radius)
    
    # Linear cluster that is closest to one of the estimated side points (ignoring z)
    # but must me within r = 15 cm and almost parallel

    closest_cluster_distance = 100000000.0
    closest_cluster = None
    closest_cluster_center = None
    
    for j, side in enumerate(indices_bothsides):
        for sidecluster, sideclustercenter, _, eigenvects_side in cluster_generator(pointcloud[side], min_points):
            
            # Check distance
            sidecluster_distance = np.linalg.norm(sideclustercenter[:2] - bothsides[j][:2]) # only xy
            if sidecluster_distance < thresh_side_distance and sidecluster_distance < closest_cluster_distance:
                # Check if parallel
                cos_angle = np.abs(np.dot(eigenvects[:,0], eigenvects_side[:,0]))
                if cos_angle > 0.9:  
                    closest_cluster = sidecluster
                    closest_cluster_distance = sidecluster_distance
                    closest_cluster_center = sideclustercenter

    return closest_cluster, closest_cluster_center

- ohne: 0.66
- linearity und on top
    - mit compile 1.16
    - dann 0.65
- die anderen mit object mode
    - mit compile 0.92s
    - dann 0.66 (langsamer!)

In [48]:
start = time.time()

for i in range(seed_point_count):
    for cluster, clustercenter, eigenvals, eigenvects in cluster_generator(xyz[indices[i]], min_points=min_points):
        pass

print(f"Time: {time.time() - start:.2f} s")

Time: 1.09 s


In [49]:
def check_seedpoints(indices, tree, gauge, min_points):
    center_points = []
    for i in range(seed_point_count):
        for cluster, clustercenter, eigenvals, eigenvects in cluster_generator(xyz[indices[i]], min_points=min_points):
            pair_cluster, pair_center = check_cluster_for_railpair(clustercenter, eigenvects, xyz, tree, gauge, min_points)
            if pair_cluster is not None:
                # lines.append(pv.Line(clustercenter, pair_center))
                center_points.append(get_central_point(clustercenter, pair_center))
    return center_points

In [50]:
# @jit(forceobj=True, looplift=True, parallel=True) # Stürzt ab
@jit(forceobj=True, parallel=True)
def check_seedpoints_numba(indices, tree, gauge, min_points):
    center_points = []
    for i in prange(seed_point_count):
        for cluster, clustercenter, eigenvals, eigenvects in cluster_generator(xyz[indices[i]], min_points=min_points):
            pair_cluster, pair_center = check_cluster_for_railpair(clustercenter, eigenvects, xyz, tree, gauge, min_points)
            if pair_cluster is not None:
                # lines.append(pv.Line(clustercenter, pair_center))
                center_points.append(get_central_point(clustercenter, pair_center))
    return center_points

- ohne 1.53 1.39  1.36 1.37
- mit stürzt ab!!!

In [54]:
start = time.time()
center_points = check_seedpoints(indices, tree, gauge, min_points)
print(time.time() - start)

1.3784897327423096


In [52]:
start = time.time()
center_points = check_seedpoints_numba(indices, tree, gauge, min_points)
print(time.time() - start)

/tmp/ipykernel_555997/3896833041.py:2: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "check_seedpoints_numba" failed type inference due to: Untyped global name 'cluster_generator': Cannot determine Numba type of <class 'function'>

File "../../../../../tmp/ipykernel_555997/3896833041.py", line 6:
<source missing, REPL/exec in use?>

  @jit(forceobj=True, parallel=True)


1.836362361907959
